In [147]:
# 2D Lid-driven Cavity
# VECTOR-POTENTIAL VORTICITY FORMULATION
# Written by Mr A. J. Brierley
# Cranfield University, Bedfordshire, UK
# 03/06/2025


In [148]:
import numpy as np
import math 
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
plt.rcParams['animation.html'] = 'html5'

In [149]:
nx = 32
ny = 32
nz = 32
lx = 1.0
ly = 1.0
lz = 1.0
dx = lx/(nx-1)
dy = ly/(ny-1)
dz = lz/(nz-1)

In [150]:
Ut = 3.2 # top wall velocity

In [151]:
# Specify initial values for the streamfunction (psi) and vorticity (Omega)
# at t = 0 on the whole domain
# Then specify conditions that will produce values at the boundaries at t = 0
# And be enforced as the solution is marched through time
psix = np.zeros([nx,ny,nz])
psiy = np.zeros([nx,ny,nz])
psiz = np.zeros([nx,ny,nz])

u = np.zeros([nx,ny,nz])
v = np.zeros([nx,ny,nz])
w = np.zeros([nx,ny,nz])

x_coords = np.linspace(0, lx, nx)
y_coords = np.linspace(0, ly, ny)
z_coords = np.linspace(0, lz, nz)

omegax = np.zeros((nx, ny, nz))
omegay = np.zeros((nx, ny, nz))
omegaz = np.zeros((nx, ny, nz))

for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            value = np.sin(np.pi * x_coords[i]) * np.sin(np.pi * y_coords[j]) * np.sin(np.pi * z_coords[k])
            omegax[i,j,k] = value
            omegay[i,j,k] = value
            omegaz[i,j,k] = value
            
# omegax = np.zeros([nx,ny,nz])
# omegay = np.zeros([nx,ny,nz])
# omegaz = np.zeros([nx,ny,nz])

In [152]:
#------------------------------------------------------
# VECTOR-POTENTIAL BOUNDARY CONDITIONS
#------------------------------------------------------
# Left wall
for j in range(ny):     # Not inclusive
    for k in range(nz):
        psix[0,j,k] = psix[1,j,k]
        psiy[0,j,k] = 0.0
        psiz[0,j,k] = 0.0

# Bottom wall
for i in range(nx):
    for z in range(nz):
        psix[i,0,k] = 0.0
        psiy[i,0,k] = psiy[i,0,k]
        psiz[i,0,k] = 0.0

# Right wall 
for j in range(ny):
    for z in range(nz):
        psix[nx-1,j,k] = psix[nx-2,j,k]  # Here, nx will try to index 13 which is outside the 0 to 13 exclusive range
        psiy[nx-1,j,k] = 0.0
        psiz[nx-1,j,k] = 0.0

# Top wall
for i in range(nx):
    for z in range(nz):
        psix[i,ny-1,k] = 0.0
        psiy[i,ny-1,k] = psiy[i,ny-2,k]
        psiz[i,ny-1,k] = 0.0

# Front wall
for i in range(nx):
    for j in range(ny):
        psix[i,j,0] = 0.0
        psiy[i,j,0] = 0.0
        psiz[i,j,0] = psiz[i,j,1]

# Back wall
for i in range(nx):
    for j in range(ny):
        psix[i,j,nz-1] = 0.0
        psiy[i,j,nz-1] = 0.0
        psiz[i,j,nz-1] = psiz[i,j,nz-2]

In [153]:
#------------------------------------------------
# VORTICITY BOUNDARY CONDITIONS 
#------------------------------------------------
# Left wall
for j in range(ny):
    for k in range(nz):
        omegax[0,j,k] = 0.0
        omegay[0,j,k] = - w[1,j,k]/dx
        omegaz[0,j,k] = v[1,j,k]/dx
# Right wall
for j in range(ny):
    for k in range(nz):
        omegax[nx-1,j,k] = 0.0
        omegay[nx-1,j,k] = w[nx-2,j,k]/dx
        omegaz[nx-1,j,k] = -v[nx-2,j,k]/dx
# Lower wall 
for i in range(nx):  # starting from 0 and exclusive
    for k in range(nz):
        omegax[i,0,k] = w[i,1,k]/dy
        omegay[i,0,k] = 0.0
        omegaz[i,0,k] = -u[i,1,k]/dy
# Front wall
for i in range(nx):
    for j in range(ny):
        omegax[i,j,0] = -v[i,j,1]/dz
        omegay[i,j,0] = u[i,j,1]/dz
        omegaz[i,j,0] = 0.0
# Back wall 
for i in range(nx):
    for j in range(ny):
        omegax[i,j,nz-1] = v[i,j,nz-2]/dz
        omegay[i,j,nz-1] = -u[i,j,nz-2]/dz
        omegaz[i,j,nz-1] = 0.0
# Top wall 
for i in range(nx):
    for k in range(nz-1):  # NOTE: dodgy bit
        # BOX
        omegax[i,ny-1,k] = -w[i,ny-2,k]/dy
        omegay[i,ny-1,k] = 0.0
        omegaz[i,ny-1,k] = -u[i,ny-1,k]/dy

        # DRIVEN. Prescribe u above on the boundary.
        # omegax[i,ny-1,k] = -w[i,ny-2,k]
        # omegay[i,ny-1,k] = (u[i,ny-1,k+1] - u[i,ny-2,k])/(2*dz)
        # omegax[i,ny-1,k] = -(u[i,ny-1,k]-u[i,ny-2,k])/(2*dx)

In [154]:
# Solution Storage
psixsol = []
psiysol = []
psizsol = []
psixsol.append
psiysol.append
psizsol.append
omegasol = []
omegasol.append(omega)

In [155]:
# Time stepping
t = 0.0  # initial time and time counter
v = 0.05  # nu
dt = min(0.25*dx*dx/v, 4*v/Ut/Ut)
tend = 1000*dt
print('dt =', dt, 's')
print('Re =', Ut*lx/v)

dt = 0.01953125 s
Re = 64.0


In [156]:
#-----------------------------------------------------
# SOLVE THREE VECTOR-POTENTIAL POISSON EQUATIONS
#-----------------------------------------------------
maxIt = 100
err = 1e5
tol = 1e-3
beta = 1.5
# Start main time loop
while t < tend:
    it = 0
    while err > tol and it < maxIt:
        psikx = psix.copy() # Stores the previous value
        psiky = psiy.copy()
        psikz = psiz.copy()
        # Solve vector-potential Poisson for psix
        for i in range(1,nx-1):
            for j in range(1,ny-1):
                for k in range(1,nz-1):
                    psix[i,j,k] = dx**2*dy**2*dz**2*omegax[i,j,k] + \
                                    dy**2*dz**2*(psix[i+1,j,k] + psix[i-1,j,k]) + \
                                    dx**2*dz**2*(psix[i,j+1,k] + psix[i,j-1,k]) + \
                                    dx**2*dy**2*(psix[i,j,k+1] + psix[i,j,k-1])
                    psix[i,j,k] = beta * psix[i,j,k]/(2*(dz**2*dy**2 + dx**2*dz**2 + dx**2*dy**2)) + (1-beta)*psikx[i,j,k]
    
        # Solve vector-potential Poisson for psiy
        for i in range(1,nx-1):
            for j in range(1,ny-1):
                for k in range(1,nz-1):
                    psiy[i,j,k] = dx**2*dy**2*dz**2*omegay[i,j,k] + \
                                    dy**2*dz**2*(psiy[i+1,j,k] + psiy[i-1,j,k]) + \
                                    dx**2*dz**2*(psiy[i,j+1,k] + psiy[i,j-1,k]) + \
                                    dx**2*dy**2*(psiy[i,j,k+1] + psiy[i,j,k-1])
                    psiy[i,j,k] = beta * psiy[i,j,k]/(2*(dz**2*dy**2 + dx**2*dz**2 + dx**2*dy**2)) + (1-beta)*psiky[i,j,k]
    
        # Solve vector-potential Poisson for psiz
        for i in range(1,nx-1):
            for j in range(1,ny-1):
                for k in range(1,nz-1):
                    psiz[i,j,k] = dx**2*dy**2*dz**2*omegaz[i,j,k] + \
                                    dy**2*dz**2*(psiz[i+1,j,k] + psiz[i-1,j,k]) + \
                                    dx**2*dz**2*(psiz[i,j+1,k] + psiz[i,j-1,k]) + \
                                    dx**2*dy**2*(psiz[i,j,k+1] + psiz[i,j,k-1])
                    psiz[i,j,k] = beta * psiz[i,j,k]/(2*(dz**2*dy**2 + dx**2*dz**2 + dx**2*dy**2)) + (1-beta)*psikz[i,j,k]
    
        err_x = np.linalg.norm(psix.ravel() - psikx.ravel())
        err_y = np.linalg.norm(psiy.ravel() - psiky.ravel())
        err_z = np.linalg.norm(psiz.ravel() - psikz.ravel())
        err = max(err_x, err_y, err_z)
        print(err)
        it = it + 1
    
    t = t + dt
    
print(f"psix = ", psix)
print(f"psiy = ", psiy)
print(f"psiz = ", psiz)
print(f"iteration =", it)
print(f"err =", err)

X, Y, Z = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')
omegax_analytical = np.sin(np.pi * X) * np.sin(np.pi * Y) * np.sin(np.pi * Z)
analytical_psix = -omegax_analytical / (3 * np.pi**2)

plt.figure(figsize=(8, 6))
plt.contourf(X[:, :, nz//2], Y[:, :, nz//2], psix[:, :, nz//2], cmap='viridis', levels=20)
plt.colorbar(label='psix')
plt.title('Numerical psix at z = Lz/2')
plt.xlabel('x')
plt.ylabel('y')
plt.savefig('psix_numerical.png')
plt.close()

# Visualize analytical solution for comparison
plt.figure(figsize=(8, 6))
plt.contourf(X[:, :, nz//2], Y[:, :, nz//2], analytical_psix[:, :, nz//2], cmap='viridis', levels=20)
plt.colorbar(label='Analytical psix')
plt.title('Analytical psix at z = Lz/2')
plt.xlabel('x')
plt.ylabel('y')
plt.savefig('psix_analytical.png')
plt.close()

# Visualize difference (error) between numerical and analytical
plt.figure(figsize=(8, 6))
plt.contourf(X[:, :, nz//2], Y[:, :, nz//2], psix[:, :, nz//2] - analytical_psix[:, :, nz//2], cmap='RdBu', levels=20)
plt.colorbar(label='Error (psix - analytical)')
plt.title('Error in psix at z = Lz/2')
plt.xlabel('x')
plt.ylabel('y')
plt.savefig('psix_error.png')
plt.close()

0.09365130999147193
0.05401083587523957
0.030402717655308102
0.016066351988462495
0.007920146750136685
0.003949420306048724
0.0024083972591034688
0.001651688107148238
0.001011631682655969
0.0005346097341471884
psix =  [[[0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]]

 [[0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.0043676  0.00751275 0.0086216  0.00744518 0.00429618
   0.        ]
  [0.         0.00751275 0.01293406 0.0148899  0.01289871 0.0074699
   0.        ]
  [0.